In [ ]:
from helpers_lstm import *
import seaborn as sns
from tqdm import trange
import sklearn


In [ ]:
df_train = pd.read_csv('./train-valid-splits-video/train.csv')
df_valid = pd.read_csv('./train-valid-splits-video/valid.csv')
#train_loader = load_data(df_train, 32, 16)
valid_loader = load_data(df_valid, 32, 16)

In [ ]:
device = torch.device('cuda')

cnn_encoder = ResCNNEncoder().to(device)
adaptive_pool = AdaptiveConcatPool2d()
cnn_encoder.resnet[8] = adaptive_pool

rnn_decoder = DecoderRNNattention(batch_size=48).to(device)

crnn_params, cnn_encoder, rnn_decoder = parallelize_model(cnn_encoder, rnn_decoder)


In [ ]:
rnn_decoder.load_state_dict(torch.load('/media/scratch/astamoulakatos/save-model-lstm/rnn_decoder_epoch_attention_freezed_5.pth'), strict=False)

In [ ]:
cnn_encoder.load_state_dict(torch.load('/media/scratch/astamoulakatos/save-model-lstm/cnn_encoder_epoch_attention_5.pth'), strict=False)

In [ ]:
rnn_decoder.eval()
cnn_encoder.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for X, y in valid_loader:
        X, y = X.to(device), y.to(device).view(-1, )
        output = rnn_decoder(cnn_encoder(X))
        X = X.to(device)
        y = Variable(y.float()).to(device) 
        X = X.permute(0,2,1,3,4)
        y = y.squeeze(dim=1)
        y = y.float()
        output = model(X)
        y = y.detach().cpu()
        #loss = criterion(output, y)
        preds = torch.sigmoid(output)
        preds = preds.to(torch.float32) s
        preds = preds.detach().cpu()
        y_pred.append(preds)
        y_true.append(y)

In [ ]:
y_tr = np.vstack([t.__array__() for tensor in y_true for t in tensor])

In [ ]:
y_pr = np.vstack([t.__array__() for tensor in y_pred for t in tensor])

In [ ]:
thresholds = nsea_compute_thresholds(y_tr, y_pr)

In [ ]:
thresholds

In [ ]:
acc = accuracy_score(y_tr, y_pr)
f1 = f1_score(y_tr, y_pr, average="samples")

In [ ]:
new_compute_metrics(y_tr, y_pr, thresholds)